# Support Vector Regression - Experimento

Este componente treina um modelo Support Vector Regression usando [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê diversas ferramentas para montagem de modelo, pré-processamento de dados, seleção e avaliação de modelos, e muitos outros utilitários.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

In [ ]:
# parameters
dataset = "/tmp/data/boston.csv" #@param {type:"string"}
target = "medv" #@param {type:"feature", label:"Atributo alvo", description: "Seu modelo será treinado para prever os valores do alvo."}

# selected features to perform the model
filter_type = "remover" #@param ["incluir","remover"] {type:"string",multiple:false,label:"Modo de seleção das features",description:"Se deseja informar quais features deseja incluir no modelo, selecione a opção 'incluir'. Caso deseje informar as features que não devem ser utilizadas, selecione 'remover'. "}
model_features = "" #@param {type:"feature",multiple:true,label:"Features para incluir/remover no modelo",description:"Seu modelo será feito considerando apenas as features selecionadas. Caso nada seja especificado, todas as features serão utilizadas"}

# features to apply Ordinal Encoder
ordinal_features = "" #@param {type:"feature",multiple:true,label:"Features para fazer codificação ordinal", description: "Seu modelo utilizará a codificação ordinal para as features selecionadas. As demais features categóricas serão codificadas utilizando One-Hot-Encoding."}

# hyperparameters
kernel = "rbf" #@param ["linear", "poly", "rbf", "sigmoid", "precomputed"] {type:"string", label:"Kernel", description:"Especifica o tipo de kernel a ser usado no algoritmo"}
C = 1.0 #@param {type:"number", label:"Regularização", description:"A força da regularização é inversamente proporcional a C. Deve ser positivo. Penalidade é l2²"}
degree = 3 #@param {type:"integer", label:"Grau", description:"Grau da função polinomial do kernel ('poly'). Ignorado por outros kernels"}
gamma = "auto" #@param ["scale", "auto"] {type: "string", label:"Gama", description:"Coeficiente de kernel para 'rbf', 'poly' e 'sigmoid'"} 
max_iter = -1 #@param {type:"integer", label:"Iteração", description:"Limite fixo nas iterações no solver, ou -1 sem limite"}

# Plots to ignore
plots_ignore = [""] #@param ["Dados de Teste", "Erro da Regressão", "Diferença do Erro", "Diferença Ordenada do Erro", "Erro Absoluto", "Probabilidade do Erro", "Erro de Segmentos", "Tabelas de Dados"] {type:"string",multiple:true,label:"Gráficos a serem ignorados", description: "Diversos gráficos são gerados ao executar o treinamento e validação do modelo, selecione quais não devem ser gerados."}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import pandas as pd

df = pd.read_csv(dataset)

## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Remoção de linhas com valores faltantes no atributo alvo

Caso haja linhas em que o atributo alvo contenha valores faltantes, é feita a remoção dos casos faltantes.

In [ ]:
df.dropna(subset=[target], inplace=True)
y = df[target].to_numpy()

## Filtragem das features 

Seleciona apenas as features que foram declaradas no parâmetro model_features. Se nenhuma feature for especificada, todo o conjunto de dados será utilizado para a modelagem.

In [ ]:
columns_to_filter = columns

if len(model_features) >= 1:

    if filter_type == "incluir":
        columns_index = (np.where(np.isin(columns, model_features)))[0]
        columns_index.sort()
        columns_to_filter = columns[columns_index]
        featuretypes = featuretypes[columns_index]
    else:
        columns_index = (np.where(np.isin(columns, model_features)))[0]
        columns_index.sort()
        columns_to_filter = np.delete(columns, columns_index)
        featuretypes = np.delete(featuretypes, columns_index)

# keep the features selected
df_model = df[columns_to_filter]
X = df_model.to_numpy()

## Divide dataset em subconjuntos de treino e teste

Subconjunto de treino: amostra de dados usada para treinar o modelo.<br>
Subconjunto de teste: amostra de dados usada para fornecer uma avaliação imparcial do treinamento do modelo no subconjunto de dados de treino.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

## Configuração dos atributos

In [ ]:
from platiagro.featuretypes import NUMERICAL

# Selects the indexes of numerical and non-numerical features
numerical_indexes = np.where(featuretypes == NUMERICAL)[0]
non_numerical_indexes = np.where(~(featuretypes == NUMERICAL))[0]

# Selects non-numerical features to apply ordinal encoder or one-hot encoder
ordinal_features = np.array(ordinal_features)

non_numerical_indexes_ordinal = np.where(
    ~(featuretypes == NUMERICAL) & np.isin(columns_to_filter, ordinal_features)
)[0]

non_numerical_indexes_one_hot = np.where(
    ~(featuretypes == NUMERICAL) & ~(np.isin(columns_to_filter, ordinal_features))
)[0]

# After the step handle_missing_values,
# numerical features are grouped in the beggining of the array
numerical_indexes_after_handle_missing_values = np.arange(len(numerical_indexes))

non_numerical_indexes_after_handle_missing_values = np.arange(
    len(numerical_indexes), len(featuretypes)
)

ordinal_indexes_after_handle_missing_values = non_numerical_indexes_after_handle_missing_values[
    np.where(np.isin(non_numerical_indexes, non_numerical_indexes_ordinal))[0]
]

one_hot_indexes_after_handle_missing_values = non_numerical_indexes_after_handle_missing_values[
    np.where(np.isin(non_numerical_indexes, non_numerical_indexes_one_hot))[0]
]

## Treina um modelo usando sklearn.svm.SVR

In [ ]:
from category_encoders.one_hot import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR

pipeline = Pipeline(
    steps=[
        (
            "handle_missing_values",
            ColumnTransformer(
                [
                    ("imputer_mean", SimpleImputer(strategy="mean"), numerical_indexes),
                    (
                        "imputer_mode",
                        SimpleImputer(strategy="most_frequent"),
                        non_numerical_indexes,
                    ),
                ],
                remainder="drop",
            ),
        ),
        (
            "handle_categorical_features",
            ColumnTransformer(
                [
                    (
                        "feature_encoder_ordinal",
                        OrdinalEncoder(),
                        ordinal_indexes_after_handle_missing_values,
                    ),
                    (
                        "feature_encoder_onehot",
                        OneHotEncoder(),
                        one_hot_indexes_after_handle_missing_values,
                    ),
                ],
                remainder="passthrough",
            ),
        ),
        (
            "estimator",
            SVR(kernel=kernel, degree=degree, gamma=gamma, C=C, max_iter=max_iter),
        ),
    ]
)

pipeline.fit(X_train, y_train)

features_after_pipeline = np.concatenate(
    (columns_to_filter[numerical_indexes], columns_to_filter[non_numerical_indexes])
)

## Avaliação de desempenho

São calculadas diferentes métricas para avaliar o desempenho do modelo treinado.

In [ ]:
from sklearn.metrics import (
    r2_score, 
    mean_absolute_error, 
    mean_squared_error,
    mean_poisson_deviance
)

# uses the model to make predictions on the Test Dataset
y_pred = pipeline.predict(X_test)

# computes metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

## Salva métricas

Utiliza a função `save_metrics` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar métricas. Por exemplo: `accuracy`, `precision`, `r2_score`, `custom_score` etc.<br>

In [ ]:
from platiagro import save_metrics

save_metrics(r2_score=r2, mae_score=mae, mse_score=mse)

## Visualiza resultados

São criados diversos gráficos para apresentar os dados e o comportamento dos erros ocorridos nos dados de teste.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from platiagro.plotting import plot_regression_data

if "Dados de Teste" not in plots_ignore:

    plot_regression_data(pipeline, columns_to_filter, X_train, y_train, X_test, y_test, y_pred)
    plt.show()

In [ ]:
from platiagro.plotting import plot_regression_error

if "Erro da Regressão" not in plots_ignore:
    
    plot_regression_error(y_test, y_pred)
    plt.show()

In [ ]:
from platiagro.plotting import plot_prediction_diff

if "Diferença do Erro" not in plots_ignore:

    plot_prediction_diff(y_test, y_pred)
    plt.show()

In [ ]:
from platiagro.plotting import plot_sorted_prediction_diff

if "Diferença Ordenada do Erro" not in plots_ignore:

    plot_sorted_prediction_diff(y_test, y_pred)
    plt.show()

In [ ]:
from platiagro.plotting import plot_absolute_error

if "Erro Absoluto" not in plots_ignore:
    
    plot_absolute_error(y_test, y_pred)
    plt.show()

In [ ]:
from platiagro.plotting import plot_probability_error

if "Probabilidade do Erro" not in plots_ignore:
    
    plot_probability_error(y_test, y_pred)
    plt.show()

In [ ]:
from platiagro.plotting import plot_segment_error

if "Erro de Segmentos" not in plots_ignore:
    
    plot_segment_error(y_test, y_pred)
    plt.show()

Visualização da tabela contendo os resultados finais

In [ ]:
from platiagro.plotting import plot_data_table

if "Tabelas de Dados" not in plots_ignore:

    df_test = pd.DataFrame(X_test, columns=columns_to_filter)
    df_test[target] = y_pred
    ax = plot_data_table(df_test)
    plt.show()

## Salva alterações no conjunto de dados

O conjunto de dados será salvo (e sobrescrito com as respectivas mudanças) localmente, no container da experimentação, utilizando a função `pandas.DataFrame.to_csv`.<br>

In [ ]:
pipeline.fit(X, y)

y_pred = pipeline.predict(X)
new_columns = "SVR" + "_prediction"
df[new_columns] = y_pred

# save dataset changes
df.to_csv(dataset, index=False)

## Salva resultados da tarefa 

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.

In [ ]:
from joblib import dump

artifacts = {
    "pipeline": pipeline,
    "columns": columns,
    "columns_to_filter": columns_to_filter,
    "new_columns": new_columns,
    "features_after_pipeline": features_after_pipeline,
}

dump(artifacts, "/tmp/data/svr.joblib")